In [4]:
import pandas
#import helpers
import numpy as np

# coding=utf-8
import os
import pandas

def save_clean_data(cleaner, X_train, y_train, X_test, name='simple'):
    path = './data/clean/' + name
    if not os.path.exists(path):
        os.makedirs(path)

    y_train.to_csv(path + '/y_train.csv')
    cleaner(X_train).to_csv(path + '/X_train.csv')
    cleaner(X_test).to_csv(path + '/X_test.csv')

def get_clean_data(cleaner_name='simple'):
	path = './data/clean/' + cleaner_name
	X_train = pandas.read_csv(path + '/X_train.csv', index_col='match_id')
	y_train = pandas.read_csv(path + '/y_train.csv', index_col='match_id')
	X_test = pandas.read_csv(path + '/X_test.csv', index_col='match_id')
	return X_train, y_train['radiant_win'], X_test

In [6]:
# Загружаем данные
train = pandas.read_csv('features.csv', index_col='match_id')
test = pandas.read_csv('features_test.csv', index_col='match_id')

# Удаляем признаки, связанные с итогами матча
train.drop(['duration', 
         'tower_status_radiant', 
         'tower_status_dire', 
         'barracks_status_radiant', 
         'barracks_status_dire'
        ], axis=1, inplace=True)

# И разделяем датасет на признаки и целевую переменную
X = train
y = train['radiant_win'].to_frame()
del train['radiant_win']

In [7]:
# Заменяем пропуски на 0
def clean(X):
    return X.fillna(0)

# В данных присутствует 11 категориальных признаков, удаляем их
def clean_category(X):
    X = clean(X)
    del X['lobby_type']
    for n in xrange(1, 6):
        del X['r{}_hero'.format(n)]
        del X['d{}_hero'.format(n)]
    return X

In [10]:
# heroes = pandas.read_csv('./data/dictionaries/heroes.csv')
# print ('Всего героев в игре:', len(heroes))

In [11]:
# Формируем "мешок слов" по героям
def hero_bag(X):
    X_pick = np.zeros((X.shape[0], len(heroes)))
    for i, match_id in enumerate(X.index):
        for p in xrange(5):
            X_pick[i, X.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
            X_pick[i, X.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

    return pandas.DataFrame(X_pick, index=X.index)

In [ ]:
save_clean_data(clean, X, y, test)

In [ ]:
save_clean_data(clean_category, X, y, test, name='clean_category')

In [ ]:
save_clean_data(hero_bag, X, y, test, name='hero_bag')